In [20]:
import pandas as pd
import numpy as np
import ast
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.core.display import display, HTML
from IPython.display import display, clear_output

In [21]:
top_5_up_df = pd.read_csv('Data/top_5_1980s_up.csv')
top_5_up_df.rename(columns={'Unnamed: 0':'id'}, inplace=True)
top_5_up_df.reset_index(inplace=True)
top_5_up_df.drop('index', axis=1, inplace=True)
top_5_up_df.set_index(['id','name'], inplace=True)
#top_5_up_df.head()
#top_5_df.shape

top_5_down_df = pd.read_csv('Data/top_5_1980s_down.csv')
top_5_down_df.rename(columns={'Unnamed: 0':'id'}, inplace=True)
top_5_down_df.reset_index(inplace=True)
top_5_down_df.drop('index', axis=1, inplace=True)
top_5_down_df.set_index(['id','name'], inplace=True)
#top_5_down_df.head()
#top_5_df.shape

top_5_df = pd.concat([top_5_up_df, top_5_down_df])
#top_5_df.shape

In [22]:
indices = pd.Series(top_5_df.index.get_level_values(1))
#indices

In [23]:
#TRY TO GET DROP DOWN LIST WITH ARTIST NAME FOR SONGS WITH SAME NAME
def num_of_artists(name):
    artists = []
    name = name.lower()
    for i in range(len(indices[indices==name])):
        idx = indices[indices == name].index[i]
        artists.append(top_5_df.iloc[idx][0])
    return artists

In [24]:
def recommendations(name, artists):
    #INPUT A SONG RETURN
    name = name.lower()
    ind = 0
    for i in range(len(indices[indices == name])):
        if(top_5_df.iloc[indices[indices == name].index[i]][0] == artists):
            ind = i
    idx = indices[indices == name].index[ind]
    #print(top_5_df.iloc[idx][0])
    
    top = top_5_df.iloc[idx]
    top_5_dict = ast.literal_eval(top.top_5)
    return top_5_dict

In [25]:
%%html
<style>

.box_style{
    width:100%;
    height:100%;
}

.head_style{
    color:green;
}

.txt_style{
    font-size:100px;
}

.btn_style{
    background-color:green;
    color:black;
    font-size: 15px;
    font-weight: bold;
}
</style>

In [26]:
song_name = widgets.Text(description='Song Name:',placeholder='Please enter a song')
song_name.add_class('txt_style')

artists = num_of_artists(song_name.value)

artist = widgets.Dropdown(
            options=artists,
            description='Artists Name',
            disbaled=False)

In [27]:
def update_song(change):
    song_name.value = change['new']
    artists = num_of_artists(song_name.value)
    artist.options = artists
song_name.observe(update_song, 'value')


In [28]:
get_top_5 = widgets.Button(
                description='Find recommended songs',
                tooltip='Find Song',
                layout=widgets.Layout(width='40%', height='50px'),
                style={'description_width': 'intitial'}
            )
        
get_top_5.add_class("btn_style")

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        try:
            top_5 = recommendations(str(song_name.value), str(artist.value))
            display(HTML(f'<h3>Here are the top 5 recommended songs for \"{song_name.value}\" by {artist.value}</h3>'))
            for key, value in top_5.items():
                url = f'https://open.spotify.com/track/{key}'
                display(HTML('<div style=font-size:large;>Song Name: <a href={}> {}</a> by: {}</div>'.format(url, value[0], value[1])))
        except IndexError:
            display(HTML(f'<h3>Sorry Song Not Found! Please Enter In Another Song</h3>'))
        
get_top_5.on_click(on_button_clicked)

btn_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

hbox_layout = widgets.Layout(margin='10px 0px 40px 0px')

hbox_result = widgets.HBox([song_name, artist], layout=hbox_layout)
vbox_result = widgets.VBox([hbox_result, get_top_5, output], layout=btn_layout)

In [29]:
text_0 = widgets.HTML(value='<h1 style=font-size:50px;>Jacob\'s Jukebox</h1>')
text_1 = widgets.HTML(value='<h1 style=color:green>Enter in a song to find the top 5 recommended songs!</h1>')
text_2 = widgets.HTML(value='<p style=font-size:20px;>Songs are recommended based on the metrics acousticness, danceability, energy,\
                      instrumentalness, liveness, loudness, speechiness, tempo, and valence using cosine similarity.</p>')

text_0.add_class('head_style')
text_2.add_class('head_style')
box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

vbox_text = widgets.VBox([text_0, text_2, text_1], layout=box_layout)
vbox_output=widgets.VBox([vbox_text, vbox_result])

In [30]:
page = widgets.HBox([vbox_output], layout=box_layout)
page.add_class("box_style")
display(page)